In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
import glob
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
from imblearn.over_sampling import RandomOverSampler, SMOTE
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import AdamW
from torch.optim.lr_scheduler import OneCycleLR
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.nn.utils import clip_grad_norm_
from collections import deque

from models import CreditScoringModel, SimplerCreditScoringModel_v1, AdvancedCreditScoringModel_v1, AdvancedCreditScoringModel_v2, AdvancedCreditScoringModel_cat
from utils import train_test_model

device = torch.device('cuda')
torch.cuda.empty_cache()

In [3]:
path_train = 'train'
path_test = 'test'
filenames_train = glob.glob(path_train + "/*.csv")
data_files_train = []

for filename in filenames_train:
    data_files_train.append(pd.read_csv(filename))

data_train = pd.concat(data_files_train, ignore_index=True)

filenames_test = glob.glob(path_test + "/*.csv")

data_files_test = []

for filename in filenames_test:
    data_files_test.append(pd.read_csv(filename))

data_test = pd.concat(data_files_test, ignore_index=True)
ids = data_test['id']

X_train = data_train.drop(['target', 'smpl', 'id'], axis=1)
y_train = data_train['target']
cat_features = set(['feature_3', 'feature_10', 'feature_12', 'feature_14', 'feature_15', 'feature_17', 'feature_20', 'feature_21', 'feature_23', 'feature_54', 'feature_60', 'feature_70', 'feature_92', 'feature_98', 'feature_101', 'feature_126', 'feature_135', 'feature_137', 'feature_142', 'feature_153', 'feature_179', 'feature_187', 'feature_198', 'feature_242', 'feature_244', 'feature_255', 'feature_270', 'feature_312', 'feature_330', 'feature_339', 'feature_341', 'feature_342', 'feature_372', 'feature_375', 'feature_383', 'feature_396', 'feature_411'])


features = X_train.columns
idx = {}

for i, feature in enumerate(features):
    if feature in cat_features:
        idx[i] = 0

for i, feature in enumerate(features):
    if i in idx.keys():
        idx[i] = len(set(X_train[feature]))

X_test = data_test.drop(['smpl'], axis=1)
X_train['feature_411'].value_counts()
for i in range(len(X_train['feature_411'])):
    if X_train['feature_411'].iloc[i] == 1:
        print(i)
        break

16


In [4]:
# model = SimplerCreditScoringModel_v1(418, droprob = 0.7).double().to(device)
# model1, train_losses, val_losses = train_test_model(model, X_train, y_train, batch_size=20000, epochs=100, learning_rate=0.001, resample = False, X_test = X_test, test_size = 0.1)

# plt.figure(figsize=(4, 4))
# plt.plot(val_losses)
# plt.show()
# torch.cuda.empty_cache()

In [5]:
# model = AdvancedCreditScoringModel_v2(418, droprob = 0.8).double().to(device)
# model2, train_losses, val_losses = train_test_model(model, X_train, y_train, batch_size=9000, epochs=50, learning_rate=0.001, resample = True, X_test = X_test, test_size = 0.2)

# plt.figure(figsize=(4, 4))
# plt.plot(val_losses)
# plt.show()
# torch.cuda.empty_cache()

In [6]:
# model = CreditScoringModel(418, droprob = 0.5).double().to(device)
# model3, train_losses, val_losses = train_test_model(model, X_train, y_train, batch_size=15000, epochs=50, learning_rate=0.001, resample = True, X_test = X_test, test_size = 0.2)

# plt.figure(figsize=(4, 4))
# plt.plot(val_losses)
# plt.show() 
# torch.cuda.empty_cache()

In [7]:
model = AdvancedCreditScoringModel_cat(418, cat_dims = idx).double().to(device)
model4, train_losses, val_losses = train_test_model(model, X_train, y_train, batch_size = 15000, epochs = 50, learning_rate = 0.001, resample = True, X_test = X_test, test_size = 0.2)

plt.figure(figsize = (4, 4))
plt.plot(val_losses)
plt.show()
torch.cuda.empty_cache()

/home/pavel/anaconda3/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


tensor(0, device='cuda:0')
3


ValueError: Categorical feature at index 410 contains values >= 3